<a href="https://colab.research.google.com/github/JuanZapa7a/semiotics/blob/main/QAT_MNIST_NOISY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Installation of Larq and necessary dependencies

In [ ]:
!pip -q install tensorflow==2.10.0
!pip -q install larq==0.13.1

import tensorflow as tf
import larq as lq

## 2. Data preparation (MNIST)

Download and prepare the MNIST dataset.

By default, each MNIST image has a shape of (28, 28), which is 2D.
However, neural networks (especially convolutional networks) typically expect 3D inputs: (height, width, channels).
Adding a channel dimension (with value 1 for grayscale) changes each image shape from (28, 28) to (28, 28, 1), which is required for most neural network layers to interpret the images correctly.
The overall shapes for the dataset become (60000, 28, 28, 1) for training and (10000, 28, 28, 1) for testing.

The MNIST dataset’s pixel values originally range from 0 to 255.
Dividing by 127.5 and then subtracting 1 maps the values to a -1 to 1 range, which can help certain models converge more quickly and maintain numerical stability. (Centering pixel values around zero often benefits neural networks as it reduces bias and helps gradient-based methods perform better.)

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1)) # (60000, 28, 28) (60000,)
test_images = test_images.reshape((10000, 28, 28, 1)) # (10000, 28, 28) (10000,)

# For binarized models, it is standard to normalize images to a range between -1 and 1.
train_images, test_images = train_images / 127.5 - 1,test_images / 127.5 - 1

print(train_images.shape, train_labels.shape)  # Debe ser (60000, 28, 28, 1), (60000,)
print(test_images.shape, test_labels.shape)    # Debe ser (10000, 28, 28, 1), (10000,)

## 5. Creation of a binarized model

The following will create a simple binarized CNN.

The quantization function
$$
q(x) = \begin{cases}
    -1 & x < 0 \\\
    1 & x \geq 0
\end{cases}
$$
is used in the forward pass to binarize the activations and the latent full precision weights. The gradient of this function is zero almost everywhere which prevents the model from learning.

To be able to train the model the gradient is instead estimated using the Straight-Through Estimator (STE)
(the binarization is essentially replaced by a clipped identity on the backward pass):
$$
\frac{\partial q(x)}{\partial x} = \begin{cases}
    1 & \left|x\right| \leq 1 \\\
    0 & \left|x\right| > 1
\end{cases}
$$

In Larq this can be done by using `input_quantizer="ste_sign"` and `kernel_quantizer="ste_sign"`.
Additionally, the latent full precision weights are clipped to -1 and 1 using `kernel_constraint="weight_clip"`.

```python
import larq as lq
import tensorflow as tf

# Define default quantization options for all layers except the first layer
kwargs = dict(
    input_quantizer="ste_sign",      # Quantizes activations using Sign-STE (binary quantization to -1 or +1)
    kernel_quantizer="ste_sign",     # Quantizes weights to binary values (-1 or +1) with Sign-STE
    kernel_constraint="weight_clip"  # Clips weights within a set range (typically -1 to +1) to stabilize training
)

# Initialize a Sequential model
model = tf.keras.models.Sequential()

# First layer: Quantized convolutional layer (only quantizing weights, not inputs)
model.add(lq.layers.QuantConv2D(
    32, (3, 3),                      # 32 filters of size 3x3
    kernel_quantizer="ste_sign",     # Quantize weights to -1 or +1
    kernel_constraint="weight_clip", # Restrict weights to a range for stability
    use_bias=False,                  # Disable bias for simplicity
    input_shape=(28, 28, 1)          # Input shape for MNIST (28x28 grayscale images)
))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))         # Downsample with max pooling
model.add(tf.keras.layers.BatchNormalization(scale=False)) # Batch normalization to stabilize activations

# Second quantized convolutional layer (quantizes both weights and activations)
model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))         # Downsample again
model.add(tf.keras.layers.BatchNormalization(scale=False))

# Third quantized convolutional layer (also quantizes both weights and activations)
model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Flatten())                    # Flatten the output for fully connected layers

# Quantized dense (fully connected) layer with 64 units
model.add(lq.layers.QuantDense(64, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))

# Final quantized dense layer for output (10 classes for MNIST)
model.add(lq.layers.QuantDense(10, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("softmax"))        # Softmax activation for classification probabilities

lq.models.summary(model)

```

To add noise to the weights in your model's layers, you can create a custom layer that wraps the original layers and modifies the weights during training. Here's how you can modify your code to inject noise into the weights:

### Modified code to add noise to the weights


### Explanation of Changes:
1. **`NoisyLayer` class**: This custom layer wraps an existing layer, such as `QuantConv2D` or `QuantDense`, and adds noise to its weights during training. The noise is Gaussian with a standard deviation defined by `noise_stddev`.
2. **Adding noise**: Noise is added directly to the trainable weights using `assign_add`. This affects the weights during the forward pass.
3. **Integration in the model**: All relevant layers are wrapped with instances of `NoisyLayer`.


**Note**: The primary reason to use only the standard deviation (𝜎) when modeling noise in neural networks is that most common noise sources in hardware, like thermal or electronic noise, are typically assumed to follow a Gaussian distribution (normal distribution) with a mean (𝜇) of zero.

In [ ]:
import larq as lq
import tensorflow as tf

# Define a custom layer to add noise to weights
class NoisyLayer(tf.keras.layers.Layer):
    def __init__(self, wrapped_layer, noise_mean=0.0, noise_stddev=0.05, **kwargs):
        super().__init__(**kwargs)
        self.wrapped_layer = wrapped_layer  # Original layer to wrap
        self.noise_mean = noise_mean        # Mean of the noise
        self.noise_stddev = noise_stddev    # Standard deviation of the noise

    def call(self, inputs):
        # Apply noise during both training and validation
        for weight in self.wrapped_layer.trainable_weights:
            noise = tf.random.normal(
                shape=tf.shape(weight),
                mean=self.noise_mean,
                stddev=self.noise_stddev
            )
            noisy_weight = weight + noise  # Add noise to weights
            # Apply the noisy weight to the layer
            weight.assign(noisy_weight)  # Assign the noisy weights directly

        return self.wrapped_layer(inputs)  # Apply the layer logic

# Define default quantization options for quantized layers
kwargs = dict(
    input_quantizer="ste_sign",
    kernel_quantizer="ste_sign",
    kernel_constraint="weight_clip"
)

# Initialize a Sequential model
model = tf.keras.models.Sequential()

# First convolutional layer with noise on weights
model.add(NoisyLayer(
    lq.layers.QuantConv2D(
        32, (3, 3),
        kernel_quantizer="ste_sign",
        kernel_constraint="weight_clip",
        use_bias=False,
        input_shape=(28, 28, 1)
    ),
    noise_stddev=0.05  # Standard deviation of noise
))

model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

# Second convolutional layer with noise on weights
model.add(NoisyLayer(
    lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs),
    noise_stddev=0.05
))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

# Third convolutional layer with noise on weights
model.add(NoisyLayer(
    lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs),
    noise_stddev=0.05
))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Flatten())

# Dense layer with noise on weights
model.add(NoisyLayer(
    lq.layers.QuantDense(64, use_bias=False, **kwargs),
    noise_stddev=0.05
))
model.add(tf.keras.layers.BatchNormalization(scale=False))

# Output layer with noise on weights
model.add(NoisyLayer(
    lq.layers.QuantDense(10, use_bias=False, **kwargs),
    noise_stddev=0.05
))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("softmax"))

# Display the model summary
lq.models.summary(model)

In [ ]:
# Compile the model with an optimizer and loss function
model.compile(
    optimizer='adam',                      # Adam optimizer is often effective for training binarized networks
    loss="sparse_categorical_crossentropy",# Cross-entropy loss for classification
    metrics=["accuracy"]                   # Accuracy as the evaluation metric
)

# Train the model where QAT takes place as the model learns to optimize quantized weights and activations.
history = model.fit(
    train_images, train_labels,
    epochs=6,                           # Number of training epochs
    batch_size=64,                      # Batch size for training
    validation_data=(test_images, test_labels) # Evaluate on test set after each epoch
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_images, test_labels)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values for both models
plt.figure(figsize=(12, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy (Binarized)')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy (Binarized)')
plt.plot(history_normal.history['accuracy'], label='Train Accuracy (Normal)')
plt.plot(history_normal.history['val_accuracy'], label='Validation Accuracy (Normal)')
plt.title('Model Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss (Binarized)')
plt.plot(history.history['val_loss'], label='Validation Loss (Binarized)')
plt.plot(history_normal.history['loss'], label='Train Loss (Normal)')
plt.plot(history_normal.history['val_loss'], label='Validation Loss (Normal)')
plt.title('Model Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()


Visualize the full training history

1. **Training and Validation Accuracy**: Shows how well the model performs on the training and validation sets in terms of classification accuracy.
2. **Training and Validation Loss**: Displays the loss values over epochs, providing insight into how well the model is learning and whether it might be overfitting.